In [1]:
#import modules
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
#See max columns in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Read the CSV file with specified data types and skipped first 4 rows
vacant_lots_df = pd.read_csv("data/southern_dallas_progress_export_20230906.csv")
vacant_lots_df.head()


,Parcel ID,Adddress,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Location,Owner Property Count,Public Owner,City or County Owned,Owner is Business,Owner Occupied,Suspected Heir Property,Bank Owned Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Vacant Since,Land Size,Unnamed: 26,Unnamed: 27
0,100561000000,"1109 WOOD ST, DALLAS, TX",BUNTING PARTNERS LP,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1956.0,In City,20,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75202,Central Area - CA-1(A)-Central Area,District 14,2022-04-01,0.124653,NaN,NaN
1,101971000000,"1908 ELM ST, DALLAS, TX",JARVIS BARNEY W JR TR & MARY LOU TR,Yes,Structure,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1956.0,In City,1,NaN,No,No,Unknown,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2023-06-01,0.122321,NaN,NaN
2,101995000000,"1913 COMMERCE ST, DALLAS, TX",DALLAS CITY OF,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1956.0,In City,2926,City of Dallas,Yes,No,No,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2022-04-01,0.117204,NaN,NaN
3,103702000000,"2523 MAIN ST, DALLAS, TX",WESTDALE MAIN LTD,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1925.0,In City,90,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.126854,NaN,NaN
4,103720000000,"2619 MAIN ST, DALLAS, TX",AP DEEP ELLUM LLC,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1925.0,Out of State,43,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.114955,NaN,NaN


In [3]:
#To see the rows before dropping duplicates
old_dim = vacant_lots_df.shape
old_dim

(39645, 28)

In [4]:
# Remove duplicates based on all columns
vacant_lots_df = vacant_lots_df.drop_duplicates(subset=['Parcel ID'])


In [5]:
#Rows after duplicates have been dropped
new_dim = vacant_lots_df.shape
new_dim
print(f"Row count reduced from {old_dim[0]} to {new_dim[0]}")

Row count reduced from 39645 to 37465


In [6]:
# Recode 0 as NA in the "Year Built" column
vacant_lots_df['Year Built'] = np.where(vacant_lots_df['Year Built'] <= 222, np.nan, vacant_lots_df['Year Built'])
vacant_lots_df['Year Built'].value_counts()

Year Built
2022.0    1740
2021.0    1220
1950.0     246
1955.0     209
1953.0     199
1952.0     198
1951.0     192
1954.0     180
1948.0     160
1940.0     149
1949.0     148
2005.0     139
1956.0     139
1947.0     135
1960.0     125
1959.0     123
1925.0     117
1958.0     116
1945.0     108
1979.0      98
1982.0      97
1946.0      92
1983.0      91
1962.0      90
1961.0      88
1970.0      86
1957.0      82
2020.0      81
1930.0      80
1984.0      80
1985.0      75
2004.0      75
1971.0      73
1964.0      73
2018.0      70
1963.0      70
1965.0      69
1935.0      69
1920.0      60
1981.0      60
1980.0      56
1972.0      55
1926.0      54
1967.0      54
1941.0      54
1969.0      52
2019.0      50
2016.0      49
1973.0      49
1974.0      48
1968.0      47
1977.0      44
1978.0      43
1986.0      42
2003.0      41
1976.0      39
1966.0      39
2006.0      37
1938.0      37
1924.0      36
2017.0      36
1939.0      35
2023.0      35
1921.0      33
1923.0      31
1987.0      31

In [7]:
#Creates field Property Age Years in df

#Assign a variable to the current year
current_year = 2023

# Calculate "Property Age Years" as the difference between 2023 and Year Built
vacant_lots_df['Property Age Years'] = current_year - vacant_lots_df['Year Built']

# Assign the calculated columns to variables for potential future use
property_age_years = vacant_lots_df['Property Age Years']


In [8]:
#Creates field Vacancy Days in df

# Convert today's date to a Timestamp object
today = pd.Timestamp(datetime.today().date())

# Convert the 'Vacant Since' column to a Datetime column 
vacant_lots_df['Vacant Since'] = pd.to_datetime(vacant_lots_df['Vacant Since'])

# Calculate "Vacancy Days" as the difference between today's date and "Vacant Since" date
vacant_lots_df['Vacancy Days'] = (today - vacant_lots_df['Vacant Since']).dt.days

# Assign the calculated columns to variables for potential future use
vacancy_days = vacant_lots_df['Vacancy Days']

In [9]:

#For ending set of 3 questions use a function to map Single Family,Multi, and Commercial
def zoning_triple_category(row):
    zoning=row['Zoning']
    
    if pd.notna(zoning):#check if thed value is not NaN
        if "Single Family" in zoning or "Mobile Home" in zoning:
            return "Single Family"
        elif any(keyword in zoning for keyword in ["Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Multifamily"
        elif any(keyword in zoning for keyword in ["Commercial", "Commerical", "commercial"]):
            return "Commercial"
        return "Other" #Return "Other" for NaN or missing values
vacant_lots_df['Zoning Category 3'] = vacant_lots_df.apply(zoning_triple_category, axis=1)  

In [10]:
# Check if "CHURCH" is a partial match in the "Current Use" column and create the "Owner is Church" column
vacant_lots_df['Owner is Church']=vacant_lots_df['Current Use'].str.contains('CHURCH', case=False, na=False)

# Map the True/False values to 'Yes' and 'No' in the "Owner is Church" column
vacant_lots_df['Owner is Church'] = vacant_lots_df['Owner is Church'].map({True: 'Yes', False: 'No'})

owner_is_church=vacant_lots_df["Owner is Church"]

In [11]:

#Checks strings in zoning column and maps them
def classify_zoning_category(row):
    zoning = row['Zoning']
    
    if pd.notna(zoning):  # Check if the value is not NaN
        if "Central Area" in zoning:
            return "Central Area"
        elif "Retail" in zoning or "Neighborhood Service" in zoning:
            return "Retail"
        elif any(keyword in zoning for keyword in ["Single Family", "Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Residential"
        elif "Office" in zoning:
            return "Office"
        elif "Industrial" in zoning:
            return "Industrial"
        elif any(keyword in zoning for keyword in ["Commercial", "Commerical", "commercial"]): # Handle the typo
            return "Commercial"
    return "Other"  # Return "Other" for NaN or missing values

# Apply the function to create the "Zoning Category" column
vacant_lots_df['Zoning Category'] = vacant_lots_df.apply(classify_zoning_category, axis=1)

In [12]:
# Define a function to apply the classification logic
def classify_owner(row):
    if pd.notna(row['Public Owner']):
        return row['Public Owner']
    elif row['Public Owner'] == 'City of Dallas':
        return 'City of Dallas'
    elif row['Public Owner'] == 'City of Dallas Housing Authority':
        return 'City of Dallas Housing Authority'
    elif row['Public Owner'] == 'County of Dallas':
        return 'County of Dallas'  
    elif row['Owner is Church'] == 'Yes':
        return 'Church'
    elif row['Owner is Business'] == 'Yes':
        return 'Business'
    elif row['Bank Owned Property'] == 'Yes':
        return 'Bank'
    else:
        return 'Private Owner'

# Apply the function to create the "Owner" column
vacant_lots_df['Owner'] = vacant_lots_df.apply(classify_owner, axis=1)
   

In [13]:
# Selecting columns of interest and filtering by 'Property Type' == 'Structure'
structure_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Structure']

# Selecting columns of interest and filtering by 'Property Type' == 'Land' for land dataframe
land_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Land']

# Filter residential properties
residential_properties = vacant_lots_df[vacant_lots_df['Zoning Category'] == 'Residential']

# Filter commercial properties used Zone Category 3 column for zoned single family residential, commercial and multifamily questions
commercial_properties = vacant_lots_df[vacant_lots_df['Zoning Category 3'] == 'Commercial']


In [14]:
vacant_lots_df.head()

,Parcel ID,Adddress,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Location,Owner Property Count,Public Owner,City or County Owned,Owner is Business,Owner Occupied,Suspected Heir Property,Bank Owned Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Vacant Since,Land Size,Unnamed: 26,Unnamed: 27,Property Age Years,Vacancy Days,Zoning Category 3,Owner is Church,Zoning Category
0,100561000000,"1109 WOOD ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,20,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75202,Central Area - CA-1(A)-Central Area,District 14,2022-04-01,0.124653,NaN,NaN,NaN,537.0,Other,No,Central Area
1,101971000000,"1908 ELM ST, DALLAS, TX",Private Owner,Yes,Structure,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,1,NaN,No,No,Unknown,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2023-06-01,0.122321,NaN,NaN,NaN,111.0,Other,No,Other
2,101995000000,"1913 COMMERCE ST, DALLAS, TX",City of Dallas,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1956.0,In City,2926,City of Dallas,Yes,No,No,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2022-04-01,0.117204,NaN,NaN,NaN,537.0,Other,No,Other
3,103702000000,"2523 MAIN ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1925.0,In City,90,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.126854,NaN,NaN,NaN,537.0,Other,No,Other
4,103720000000,"2619 MAIN ST, DALLAS, TX",Business,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,NaN,1925.0,Out of State,43,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75226,Other - PD-269,District 2,2022-04-01,0.114955,NaN,NaN,NaN,537.0,Other,No,Other


In [15]:
vacant_lots_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37465 entries, 0 to 39644
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Parcel ID                               37465 non-null  object        
 1   Adddress                                37465 non-null  object        
 2   Owner                                   37465 non-null  object        
 3   Is Vacant Property                      37465 non-null  object        
 4   Property Type                           36235 non-null  object        
 5   Current Use                             37465 non-null  object        
 6   Year Built                              9118 non-null   float64       
 7   Average Year Built By Council District  37444 non-null  float64       
 8   Owner Location                          37465 non-null  object        
 9   Owner Property Count                    37465 non-null 

In [16]:
vacant_lots_df.isnull().agg(['sum'])

,Parcel ID,Adddress,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Location,Owner Property Count,Public Owner,City or County Owned,Owner is Business,Owner Occupied,Suspected Heir Property,Bank Owned Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Vacant Since,Land Size,Unnamed: 26,Unnamed: 27,Property Age Years,Vacancy Days,Zoning Category 3,Owner is Church,Zoning Category
sum,0,0,0,0,1230,0,28347,21,0,0,34810,0,0,0,0,0,32770,32770,0,31584,31584,0,7247,21,7,0,37465,37465,28347,7,7247,0,0


In [17]:
vacant_lots_df.columns

Index(['Parcel ID', 'Adddress', 'Owner', 'Is Vacant Property', 'Property Type',
       'Current Use', 'Year Built', 'Average Year Built By Council District',
       'Owner Location', 'Owner Property Count', 'Public Owner',
       'City or County Owned', 'Owner is Business', 'Owner Occupied',
       'Suspected Heir Property', 'Bank Owned Property',
       'Taxes Delinquent Since', 'Delinquent Taxes', 'Is Tax Delinquent',
       'Code Liens - Count', 'Code Liens - Amount', 'Zip Code', 'Zoning',
       'Council District', 'Vacant Since', 'Land Size', 'Unnamed: 26',
       'Unnamed: 27', 'Property Age Years', 'Vacancy Days',
       'Zoning Category 3', 'Owner is Church', 'Zoning Category'],
      dtype='object')

In [18]:
#Sets results for dict
results = {}

In [19]:
#1.How many vacant lots are in each council district?

#Groups Council distrcits column and 
vacant_lots_counts_by_district  = land_df.groupby('Council District').size()


In [20]:
#dictonary that starts the result values
results["How many vacant lots are in each council district?"] = {
    'question_number': 1,
    'value': vacant_lots_counts_by_district.to_frame(name = 'count')
}

results["How many vacant lots are in each council district?"]


{'question_number': 1,
 'value':                   count
 Council District       
 District 1         1233
 District 10         355
 District 11         368
 District 12         187
 District 13         442
 District 14         999
 District 2         2227
 District 3         1625
 District 4         3507
 District 5         1540
 District 6         4152
 District 7         4578
 District 8         3121
 District 9          251}

In [21]:
#2 How long have the vacant lots in each council district been vacant? 
vacant_lots_stats = land_df.groupby('Council District')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_lots_stats.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [22]:

results["How long have the vacant lots in each council district been vacant?"] = {
    'question_number': 2,
    'value': vacant_lots_stats
}


results["How long have the vacant lots in each council district been vacant?"]

{'question_number': 2,
 'value':                   Count  Min Days  Median Days   Mean Days  Max Days
 Council District                                                    
 District 1         1233     111.0        537.0  515.991079     537.0
 District 10         355     111.0        537.0  519.118310     537.0
 District 11         368     111.0        537.0  517.407609     537.0
 District 12         187     111.0        537.0  530.941176     537.0
 District 13         442     111.0        537.0  527.904977     537.0
 District 14         999      84.0        537.0  523.956957     537.0
 District 2         2227     111.0        537.0  525.537494     537.0
 District 3         1624     111.0        537.0  533.735837     537.0
 District 4         3507     111.0        537.0  526.640433     537.0
 District 5         1540     111.0        537.0  532.103247     537.0
 District 6         4152     111.0        537.0  524.756503     537.0
 District 7         4578     111.0        537.0  530.48208

In [23]:
# 3 How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types

# Group by 'Council District' and 'Zoning Category', then count occurrences
#zoning_category_counts = vacant_lots_df.groupby(['Council District', 'Zoning Category']).size().unstack(fill_value=0)


In [24]:

'''results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {
    'question_number': 3,
    'value': zoning_category_counts
}

results'''

'results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {\n    \'question_number\': 3,\n    \'value\': zoning_category_counts\n}\n\nresults'

In [25]:
#3 How many vacant lots in each council district are zoned single family residential, commercial and multifamily 

# Group by 'Council District' and 'Zoning Category', then count occurrences
zoning_category_counts_3 = land_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [26]:

results["How many vacant lots in each council district are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 3,
    'value': zoning_category_counts_3
}


results["How many vacant lots in each council district are zoned single family residential, commercial and multifamily?"]

{'question_number': 3,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 1                  3           67    892            225
 District 10                10           54    205             43
 District 11                 2           22    262             82
 District 12                 0           27     96             64
 District 13                 3           71    190            152
 District 14                 1           97    635              5
 District 2                194          235   1311            111
 District 3                 56          108    766            668
 District 4                 97          271   1316           1821
 District 5                 15           42    622            861
 District 6                119           92   1875           1276
 District 7                152          284    820            382
 District 8                221           99 

In [27]:
# 4. How many vacant structures are in each council district?

# Groups Council District by structure and vacant property

# Grouping by 'Council District' and counting occurrences
vacant_structure_count = structure_df.groupby('Council District').size()

In [28]:

results["How many vacant structures are in each council district?"] = {
    'question_number': 4,
    'value': vacant_structure_count.to_frame(name = 'count')
}

results["How many vacant structures are in each council district?"]

{'question_number': 4,
 'value':                   count
 Council District       
 District 1          618
 District 10         326
 District 11         401
 District 12         105
 District 13        1009
 District 14         766
 District 2         1259
 District 3          455
 District 4         1736
 District 5          598
 District 6         1539
 District 7         1224
 District 8          896
 District 9          698}

In [29]:
#5 How long have the vacant structures in each council district been vacant? (see question 2)
vacant_structure_count_days = structure_df.groupby('Council District')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_structure_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [30]:

results["How long have the vacant structures in each council district been vacant?"] = {
    'question_number': 5,
    'value': vacant_structure_count_days
}

results["How long have the vacant structures in each council district been vacant?"]

{'question_number': 5,
 'value':                   Count  Min Days  Median Days   Mean Days  Max Days
 Council District                                                    
 District 1          618     111.0        537.0  471.372168     537.0
 District 10         326     111.0        537.0  374.585890     537.0
 District 11         401     111.0        537.0  437.563591     537.0
 District 12         105     111.0        292.0  335.552381     537.0
 District 13        1009     111.0        537.0  436.011893     537.0
 District 14         766     111.0        537.0  407.233681     537.0
 District 2         1259     111.0        537.0  436.633042     537.0
 District 3          455     111.0        537.0  463.474725     537.0
 District 4         1736     111.0        537.0  450.455645     537.0
 District 5          598     111.0        537.0  474.535117     537.0
 District 6         1539     111.0        537.0  459.574399     537.0
 District 7         1224     111.0        537.0  449.06944

In [31]:

#6 How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)

zoning_category_counts_structure = structure_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [32]:

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] = {
    'question_number': 6,
    'value': zoning_category_counts_structure
}

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] 


{'question_number': 6,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 1                  4           31    323            229
 District 10                 7           19    165             46
 District 11                 0           32    204            165
 District 12                 2           20     35             48
 District 13                 0           43    252            693
 District 14                 1           90    297             24
 District 2                 47          167    452             48
 District 3                  5           15    136            297
 District 4                 13           75    275           1371
 District 5                  8           32     93            465
 District 6                 17           60    559            620
 District 7                 29          115    100            234
 District 8                 23           15 

In [33]:
#7.How many vacant lots are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_lots_counts_by_zip = land_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_lots_counts_by_zip.index = vacant_lots_counts_by_zip.index.astype(int)

In [34]:

results["How many vacant lots are in each zip code?"] = {
    'question_number': 7,
    'value': vacant_lots_counts_by_zip.to_frame(name = 'count')
}

results["How many vacant lots are in each zip code?"]

{'question_number': 7,
 'value':           count
 Zip Code       
 75006         5
 75019        44
 75041         1
 75043        10
 75050         1
 75051         9
 75060         1
 75062         2
 75063         5
 75080        10
 75081         2
 75141         7
 75149         9
 75150        13
 75159         5
 75181         4
 75182         2
 75201       327
 75202       125
 75203      2079
 75204       455
 75205        35
 75206       231
 75207       424
 75208       835
 75209       117
 75210       993
 75211       791
 75212      2404
 75214       142
 75215      2724
 75216      1897
 75217      1790
 75218        86
 75219       315
 75220       389
 75223       821
 75224       226
 75225       102
 75226       446
 75227       592
 75228       186
 75229       336
 75230       173
 75231       154
 75232       321
 75233       170
 75234        19
 75235       315
 75236       552
 75237       275
 75238        89
 75240        94
 75241      1564
 75243       254

In [35]:
#8 How long have the vacant lots in each zip code been vacant?
vacant_zips_count_days = land_df.groupby('Zip Code')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_count_days.index = vacant_zips_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [36]:

results["How long have the vacant lots in each zip code been vacant?"] = {
    'question_number': 8,
    'value': vacant_zips_count_days
}

results["How long have the vacant lots in each zip code been vacant?"] 

{'question_number': 8,
 'value':           Count  Min Days  Median Days   Mean Days  Max Days
 Zip Code                                                    
 75006         5     537.0        537.0  537.000000     537.0
 75019        44     111.0        537.0  505.886364     537.0
 75041         1     537.0        537.0  537.000000     537.0
 75043        10     537.0        537.0  537.000000     537.0
 75050         1     537.0        537.0  537.000000     537.0
 75051         9     537.0        537.0  537.000000     537.0
 75060         1     537.0        537.0  537.000000     537.0
 75062         2     537.0        537.0  537.000000     537.0
 75063         5     537.0        537.0  537.000000     537.0
 75080        10     537.0        537.0  537.000000     537.0
 75081         2     537.0        537.0  537.000000     537.0
 75141         7     537.0        537.0  537.000000     537.0
 75149         9     256.0        537.0  505.777778     537.0
 75150        13     111.0        537.

In [37]:
#9 How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?


zoning_category_counts_zip = land_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_zip.index = zoning_category_counts_zip.index.astype(int)

In [38]:

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 9,
    'value': zoning_category_counts_zip
}

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"]

{'question_number': 9,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      4              0
 75019                       0            0      9              0
 75043                       0            0      6              0
 75051                       0            0      3              6
 75060                       0            0      1              0
 75062                       0            0      2              0
 75063                       0            0      1              0
 75080                       0            0     10              0
 75081                       0            1      1              0
 75141                       0            0      7              0
 75149                       0            0      9              0
 75150                       0            0     10              0
 75159                       0            0 

In [39]:
#10 How many vacant structures are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_structure_counts_by_zip = structure_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_structure_counts_by_zip.index = vacant_structure_counts_by_zip.index.astype(int)

In [40]:

results["How many vacant structures are in each zip code?"] = {
    'question_number': 10,
    'value': vacant_structure_counts_by_zip.to_frame(name = 'count')
}

results["How many vacant structures are in each zip code?"]

{'question_number': 10,
 'value':           count
 Zip Code       
 75006         1
 75042         1
 75063         1
 75080         1
 75081         1
 75115         1
 75150         1
 75201       145
 75202        38
 75203       399
 75204       211
 75205        76
 75206       260
 75207       187
 75208       332
 75209       317
 75210       242
 75211       354
 75212       686
 75214       278
 75215       521
 75216      1407
 75217       541
 75218       336
 75219       260
 75220       341
 75223       179
 75224       120
 75225       152
 75226        68
 75227       440
 75228       685
 75229       478
 75230       224
 75231       135
 75232       193
 75233        58
 75234        19
 75235       193
 75236        67
 75237        50
 75238       110
 75240       172
 75241       537
 75243       181
 75244        68
 75246        33
 75247        88
 75248       157
 75249        56
 75251        11
 75252         7
 75253       186
 75254        24
 75270         

In [41]:
#11 How long have the vacant structures in each zip code been vacant? (see question 8)
vacant_zips_structure_count_days = structure_df.groupby('Zip Code')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_structure_count_days.index = vacant_zips_structure_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_structure_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [42]:

results["How long have the vacant structures in each zip code been vacant?"] = {
    'question_number': 11,
    'value': vacant_zips_structure_count_days
}

results["How long have the vacant structures in each zip code been vacant?"]

{'question_number': 11,
 'value':           Count  Min Days  Median Days   Mean Days  Max Days
 Zip Code                                                    
 75006         1     537.0        537.0  537.000000     537.0
 75042         1     111.0        111.0  111.000000     111.0
 75063         1     537.0        537.0  537.000000     537.0
 75080         1     111.0        111.0  111.000000     111.0
 75081         1     537.0        537.0  537.000000     537.0
 75115         1     349.0        349.0  349.000000     349.0
 75150         1     384.0        384.0  384.000000     384.0
 75201       145     111.0        256.0  324.586207     537.0
 75202        38     111.0        293.5  325.631579     537.0
 75203       399     111.0        537.0  457.909774     537.0
 75204       211     111.0        537.0  449.947867     537.0
 75205        76     111.0        537.0  460.328947     537.0
 75206       260     111.0        537.0  439.138462     537.0
 75207       187     111.0        537

In [43]:
#12 How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)

zoning_category_counts_structure_zip = structure_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_structure_zip.index = zoning_category_counts_structure_zip.index.astype(int)

In [44]:

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 12,
    'value': zoning_category_counts_structure_zip
}

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily?"]


{'question_number': 12,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      1              0
 75042                       0            0      1              0
 75080                       0            0      0              1
 75081                       0            0      1              0
 75115                       0            0      1              0
 75150                       0            0      1              0
 75201                       0            0     95              0
 75202                       0            0     38              0
 75203                       1           28    210            159
 75204                       4           97     90              1
 75205                       0            1      2              0
 75206                       1           92    120             19
 75207                       0            0

In [45]:
#13 What is the total amount of back taxes owed on vacant land in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
#Remove vacant prop yes filter
back_taxes_total_by_district = land_df.groupby('Council District')['Delinquent Taxes'].agg(['sum'])

In [46]:

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] = {
    'question_number': 13,
    'value': back_taxes_total_by_district
}

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] 

{'question_number': 13,
 'value':                       sum
 Council District         
 District 1         973.77
 District 10        153.54
 District 11         36.67
 District 12         13.76
 District 13       3350.19
 District 14       1166.13
 District 2        2776.56
 District 3        1155.22
 District 4        2589.57
 District 5         757.51
 District 6        2774.26
 District 7        2082.89
 District 8        2404.97
 District 9         229.15}

In [47]:
#14 What is the total amount of back taxes owed on vacant land in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip = land_df.groupby('Zip Code')['Delinquent Taxes'].agg(['sum'])

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip.index = back_taxes_total_by_zip.index.astype(int)


In [48]:

results["What is the total amount of back taxes owed on vacant land in each zip code?"] = {
    'question_number': 14,
    'value': back_taxes_total_by_zip
}

results["What is the total amount of back taxes owed on vacant land in each zip code?"] 

{'question_number': 14,
 'value':               sum
 Zip Code         
 75006        0.00
 75019        0.35
 75041        0.00
 75043        0.00
 75050        0.00
 75051        3.82
 75060        0.00
 75062        0.00
 75063        0.06
 75080        0.01
 75081        0.16
 75141        0.00
 75149        0.00
 75150        0.04
 75159        0.00
 75181        0.36
 75182      194.21
 75201      670.36
 75202       47.63
 75203     1077.28
 75204      511.85
 75205       12.85
 75206      114.97
 75207       63.07
 75208      770.67
 75209      102.16
 75210      352.35
 75211      638.99
 75212     1947.52
 75214       71.07
 75215     1591.48
 75216     1572.15
 75217     1012.15
 75218        0.57
 75219      430.37
 75220     3042.38
 75223      469.60
 75224      151.16
 75225       66.36
 75226     1114.80
 75227      354.76
 75228       73.90
 75229       44.03
 75230        1.55
 75231      581.54
 75232      353.92
 75233      115.23
 75234        0.00
 75235      200.4

In [49]:
#15 What is the total amount of back taxes owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
back_taxes_total_by_district_by_structure = structure_df.groupby('Council District')['Delinquent Taxes'].agg(['sum'])


In [50]:

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"] = {
    'question_number': 15,
    'value': back_taxes_total_by_district_by_structure
}

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"]

{'question_number': 15,
 'value':                       sum
 Council District         
 District 1         595.25
 District 10        257.17
 District 11        698.71
 District 12        105.98
 District 13       3978.19
 District 14       2212.51
 District 2        1110.55
 District 3         447.95
 District 4        1815.38
 District 5         693.23
 District 6        2305.88
 District 7         877.80
 District 8         665.22
 District 9         866.64}

In [51]:
#16 What is the total amount of back taxes owed on vacant structures in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip_by_structure = structure_df.groupby('Zip Code')['Delinquent Taxes'].agg(['sum'])

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip_by_structure.index = back_taxes_total_by_zip_by_structure.index.astype(int)


In [52]:

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] = {
    'question_number': 16,
    'value': back_taxes_total_by_zip_by_structure
}

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] 

{'question_number': 16,
 'value':               sum
 Zip Code         
 75006        0.00
 75042        0.00
 75063        0.00
 75080        0.00
 75081        0.00
 75115       48.01
 75150        0.01
 75201     1042.72
 75202        0.00
 75203      410.59
 75204      299.72
 75205      179.92
 75206      377.25
 75207      206.68
 75208      719.34
 75209      940.60
 75210      199.91
 75211      256.09
 75212      536.81
 75214      456.50
 75215      293.40
 75216     1406.63
 75217      569.45
 75218      595.17
 75219      238.44
 75220     1708.26
 75223      468.43
 75224       60.67
 75225      471.83
 75226        5.69
 75227      542.68
 75228      421.59
 75229     1404.10
 75230      658.43
 75231      495.71
 75232      166.81
 75233       88.56
 75234        0.00
 75235       71.04
 75236       11.48
 75237       56.99
 75238      136.23
 75240       27.56
 75241      421.36
 75243      150.21
 75244       71.40
 75246       81.20
 75247        6.56
 75248      139.6

In [53]:
#17 What is the average age of residential properties in each council district?

#avg_age__prop_District=vacant_lots_df.groupby('Council District')['Property Age Years'].agg(['mean'])
# Calculate the average age of residential properties

residential_avg_age_prop_District = residential_properties.groupby('Council District')['Property Age Years'].agg(['mean'])

In [54]:

results["What is the average age of residential properties in each council district?"] = {
    'question_number': 17,
    'value': residential_avg_age_prop_District
}

results["What is the average age of residential properties in each council district?"]

{'question_number': 17,
 'value':                        mean
 Council District           
 District 1        65.606635
 District 10       49.925926
 District 11       37.690323
 District 12       41.790323
 District 13       27.689189
 District 14       21.179487
 District 2        27.270588
 District 3        40.400000
 District 4        54.262584
 District 5        49.231552
 District 6        23.369261
 District 7        38.503268
 District 8        30.088889
 District 9        54.450000}

In [55]:
#18 What is the average age of residential properties in each zip code?
avg_age__prop_zip = residential_properties.groupby('Zip Code')['Property Age Years'].agg(['count', 'min','median', 'mean', 'max'])
avg_age__prop_zip.columns = ['count', 'min','median', 'mean', 'max']
#Fills na values with 0
#avg_age__prop_zip=avg_age__prop_zip.fillna(0)

#converts avg age to int data type to remve decimals
avg_age__prop_zip.index=avg_age__prop_zip.index.astype(int)


In [56]:

results["What is the average age of residential properties in each zip code?"] = {
    'question_number': 18,
    'value': avg_age__prop_zip
}

results["What is the average age of residential properties in each zip code?"]

{'question_number': 18,
 'value':           count   min  median       mean    max
 Zip Code                                       
 75051         0   NaN     NaN        NaN    NaN
 75080         0   NaN     NaN        NaN    NaN
 75081         0   NaN     NaN        NaN    NaN
 75159         0   NaN     NaN        NaN    NaN
 75203       151   0.0    74.0  51.602649  103.0
 75204        65   1.0     2.0  13.553846  105.0
 75205         0   NaN     NaN        NaN    NaN
 75206        90   0.0     2.0  16.200000  103.0
 75208        63   1.0    83.0  60.619048  110.0
 75209        84   0.0     2.5  25.297619   84.0
 75210        11   1.0    52.0  52.181818  112.0
 75211       189   1.0    73.0  58.349206  113.0
 75212       355   1.0     2.0  18.529577   93.0
 75214        43   1.0    17.0  40.348837  113.0
 75215         0   NaN     NaN        NaN    NaN
 75216      1027   0.0    70.0  51.103213  109.0
 75217       405   0.0    16.0  33.261728   98.0
 75218        37   1.0    70.0  64.2

In [57]:
#19 What is the average size of residential properties in each council district?


# Group by council district and calculate the average property size
average_size_by_district = residential_properties.groupby('Council District')['Land Size'].agg(['mean'])

In [58]:

results["What is the average size of residential properties in each council district?"] = {
    'question_number': 19,
    'value': average_size_by_district
}

results["What is the average size of residential properties in each council district?"]

{'question_number': 19,
 'value':                       mean
 Council District          
 District 1        0.484867
 District 10       1.719645
 District 11       1.575737
 District 12       1.243921
 District 13       1.059755
 District 14       0.290075
 District 2        0.276805
 District 3        1.474771
 District 4        0.326351
 District 5        0.746599
 District 6        0.221535
 District 7        1.177131
 District 8        1.722044
 District 9        0.546445}

In [59]:
#20 What is the average size of residential properties in each zip code?

# Group by council district and calculate the average property size
average_size_by_zip = residential_properties.groupby('Zip Code')['Land Size'].agg(['mean'])

# Convert index (zip codes) to integers to remove the .0 decimal part
average_size_by_zip.index = average_size_by_zip.index.astype(int)


In [60]:

results["What is the average size of residential properties in each zip code?"] = {
    'question_number': 20,
    'value': average_size_by_zip
}

results["What is the average size of residential properties in each zip code?"]

{'question_number': 20,
 'value':                mean
 Zip Code           
 75051     10.151089
 75080     21.995701
 75081      0.107777
 75159      0.230563
 75203      0.229143
 75204      0.181873
 75205      0.155813
 75206      0.310266
 75208      0.276088
 75209      0.772134
 75210      1.258265
 75211      1.065608
 75212      0.196358
 75214      0.363584
 75215     41.979794
 75216      0.372572
 75217      1.220618
 75218      0.474402
 75220      0.496342
 75223      0.241798
 75224      0.628400
 75225      0.678888
 75226      0.204134
 75227      0.892788
 75228      1.484102
 75229      0.712420
 75230      0.587706
 75231      5.348340
 75232      0.911567
 75233      0.798006
 75234      0.202157
 75235      0.194761
 75236      2.962961
 75237      1.920668
 75238      0.413107
 75240      1.280653
 75241      1.382709
 75243      1.917442
 75244      0.452960
 75246      0.260339
 75248      1.150895
 75249      3.911127
 75252      0.716077
 75253      1.827022
 

In [61]:
#21 What is the average age of commercial properties in each council district?

# Group by council district and calculate the average property size by commercial prop
average_age_by_district_commercial = commercial_properties.groupby('Council District')['Property Age Years'].agg(['mean'])


In [62]:

results["What is the average age of commercial properties in each council district?"] = {
    'question_number': 21,
    'value': average_age_by_district_commercial
}

results["What is the average age of commercial properties in each council district?"]

{'question_number': 21,
 'value':                        mean
 Council District           
 District 1        56.500000
 District 10       39.166667
 District 11             NaN
 District 12             NaN
 District 13             NaN
 District 14       97.000000
 District 2        59.344828
 District 3         9.000000
 District 4        71.375000
 District 5        37.333333
 District 6        55.400000
 District 7        66.571429
 District 8        43.000000
 District 9        37.600000}

In [63]:
#22 What is the average age of commercial properties in each zip code?

average_age_by_zip_commercial = commercial_properties.groupby('Zip Code')['Property Age Years'].agg(['count', 'min','median', 'mean', 'max'])
average_age_by_zip_commercial.columns = ['count', 'min','median', 'mean', 'max']

# Convert index (zip codes) to integers to remove the .0 decimal part
average_age_by_zip_commercial.index = average_age_by_zip_commercial.index.astype(int)

In [64]:
results["What is the average age of commercial properties in each zip code?"] = {
    'question_number': 22,
    'value': average_age_by_zip_commercial
}

results["What is the average age of commercial properties in each zip code?"]

{'question_number': 22,
 'value':           count   min  median       mean    max
 Zip Code                                       
 75203         1  88.0    88.0  88.000000   88.0
 75204         4  72.0    76.5  75.750000   78.0
 75206         1  97.0    97.0  97.000000   97.0
 75207         0   NaN     NaN        NaN    NaN
 75208         4  45.0    55.5  56.000000   68.0
 75209         0   NaN     NaN        NaN    NaN
 75210         0   NaN     NaN        NaN    NaN
 75211         2   1.0    13.5  13.500000   26.0
 75212         1  75.0    75.0  75.000000   75.0
 75215         0   NaN     NaN        NaN    NaN
 75216        17   2.0    69.0  63.470588   98.0
 75217         4  12.0    43.5  38.750000   56.0
 75218         3  29.0    44.0  46.000000   65.0
 75220         1  28.0    28.0  28.000000   28.0
 75223        18   3.0    67.0  60.611111  100.0
 75224         0   NaN     NaN        NaN    NaN
 75226         1  74.0    74.0  74.000000   74.0
 75227         0   NaN     NaN      

In [65]:
#23 What is the total amount of city liens owed on vacant land in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_district = land_df.groupby('Council District')['Code Liens - Amount'].agg(['sum'])


In [66]:

results["What is the total amount of city liens owed on vacant land in each City Council District?"] = {
    'question_number': 23,
    'value': city_liens_total_by_district
}

results["What is the total amount of city liens owed on vacant land in each City Council District?"]

{'question_number': 23,
 'value':                          sum
 Council District            
 District 1         106524.86
 District 10          6386.99
 District 11          9983.31
 District 12             0.00
 District 13          3929.52
 District 14         26594.54
 District 2         276429.35
 District 3         197133.66
 District 4        2406152.93
 District 5         556304.97
 District 6         525634.76
 District 7        5437704.88
 District 8         844284.21
 District 9          10875.65}

In [67]:
#24 What is the total amount of city liens owed on vacant land in each zip code?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_zip = land_df.groupby('Zip Code')['Code Liens - Amount'].agg(['sum'])

# Convert index (zip codes) to integers to remove the .0 decimal part
city_liens_total_by_zip.index = city_liens_total_by_zip.index.astype(int)


In [68]:

results["What is the total amount of city liens owed on vacant land in each zip code?"] = {
    'question_number': 24,
    'value': city_liens_total_by_zip
}

results["What is the total amount of city liens owed on vacant land in each zip code?"]

{'question_number': 24,
 'value':                  sum
 Zip Code            
 75006           0.00
 75019           0.00
 75041           0.00
 75043           0.00
 75050           0.00
 75051        3122.46
 75060           0.00
 75062           0.00
 75063           0.00
 75080           0.00
 75081           0.00
 75141           0.00
 75149         171.41
 75150           0.00
 75159           0.00
 75181           0.00
 75182           0.00
 75201           0.00
 75202           0.00
 75203      899125.30
 75204       65197.82
 75205           0.00
 75206        6015.15
 75207        4283.29
 75208       43062.93
 75209        6726.82
 75210     1443712.44
 75211       85952.16
 75212      450094.10
 75214        7111.97
 75215     3459102.96
 75216     1671369.41
 75217      593955.57
 75218        4457.29
 75219       11869.36
 75220       36165.00
 75223      383381.83
 75224       74789.23
 75225         239.61
 75226       70617.00
 75227       91518.43
 75228       13774.09

In [69]:
#25 What is the total amount of city liens owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant structures by City Council District
city_liens_total_by_district_by_structure = structure_df.groupby('Council District')['Code Liens - Amount'].agg(['sum'])


In [70]:

results["What is the total amount of city liens owed on vacant structures in each City Council District?"] = {
    'question_number': 25,
    'value': city_liens_total_by_district_by_structure
}

results["What is the total amount of city liens owed on vacant structures in each City Council District?"]

{'question_number': 25,
 'value':                         sum
 Council District           
 District 1         50196.85
 District 10         3033.85
 District 11        32389.45
 District 12        49869.82
 District 13         9896.19
 District 14        15862.81
 District 2         67722.15
 District 3         84934.17
 District 4        826138.18
 District 5        117714.75
 District 6        145315.51
 District 7        487400.39
 District 8        213703.99
 District 9         38464.13}

In [71]:
#26 How many properties are in the name of heirship in the City of Dallas?

#Use suspected heir prop col and combine with public owner filter
# Filter the DataFrame for properties in the City of Dallas with 'Suspected Heir Property' == 'Yes'
#Would get zero combined with City of Dallas filter
dallas_heirship_properties = (vacant_lots_df['Suspected Heir Property'] == 'Yes').sum()



In [72]:

results["How many properties are in the name of heirship in the City of Dallas?"] = {
    'question_number': 26,
    'value':  pd.Series(dallas_heirship_properties).to_frame(name='count')
}

results["How many properties are in the name of heirship in the City of Dallas?"]

{'question_number': 26,
 'value':    count
 0    260}

In [73]:
#27 How many properties in the name of heirship in the City of Dallas are vacant in each zip code?

#apply Suspected heir to Yes and group by zip code
dallas_heirship_zip = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
dallas_heirship_zip.index = dallas_heirship_zip.index.astype(int)


In [74]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] = {
    'question_number': 27,
    'value': dallas_heirship_zip.to_frame(name = 'count')
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] 

{'question_number': 27,
 'value':           count
 Zip Code       
 75201         4
 75203        21
 75206         3
 75207         1
 75208         3
 75209         1
 75210        13
 75211         5
 75212        21
 75214         4
 75215        35
 75216        49
 75217        11
 75218         3
 75220         2
 75223         6
 75224         1
 75225         3
 75226         4
 75227         9
 75228         9
 75230         1
 75231         2
 75232         4
 75233         1
 75235         1
 75237         4
 75238         1
 75241        24
 75243         1
 75246         1
 75248         2
 75249         2
 75253         8}

In [75]:
#28 How many properties in the name of heirship in the City of Dallas are vacant in each council district?

#apply Suspected heir to Yes and group by council distrcit
dallas_heirship_District = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Council District').size()


In [76]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] = {
    'question_number': 28,
    'value': dallas_heirship_District.to_frame(name = 'count')
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] 

{'question_number': 28,
 'value':                   count
 Council District       
 District 1            4
 District 10           4
 District 11           2
 District 12           1
 District 13           4
 District 14           8
 District 2           13
 District 3           12
 District 4           59
 District 5           16
 District 6           26
 District 7           69
 District 8           34
 District 9            8}

In [77]:
#29 How many private owners have five or more vacant properties in the City of Dallas?
owner_vacant_property_counts = vacant_lots_df[
    (vacant_lots_df['Owner'] == 'Private Owner') &
    (vacant_lots_df['Owner Property Count'] >= 5)
]

owner_vacant_property_counts_view = owner_vacant_property_counts.shape[0]


In [78]:

results["How many private owners have five or more vacant properties in the City of Dallas?"] = {
    'question_number': 29,
    'value': pd.Series(owner_vacant_property_counts_view).to_frame(name='count')
}

results["How many private owners have five or more vacant properties in the City of Dallas?"]

{'question_number': 29,
 'value':    count
 0   7348}

In [79]:
#30 How many private owners with five or more vacant properties in the City of Dallas are in each zip code?

private_5_zip = owner_vacant_property_counts.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
private_5_zip.index = private_5_zip.index.astype(int)


In [80]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 30,
    'value': private_5_zip.to_frame(name = 'count')
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"]

{'question_number': 30,
 'value':           count
 Zip Code       
 75006         1
 75019        26
 75043         1
 75050         1
 75051         1
 75063         6
 75080         1
 75081         1
 75149         1
 75181         2
 75182         1
 75201        55
 75202        23
 75203       570
 75204        86
 75205        12
 75206        73
 75207        91
 75208       219
 75209        65
 75210       237
 75211       312
 75212       910
 75214        33
 75215       657
 75216       672
 75217       445
 75218        60
 75219        64
 75220       126
 75223       218
 75224        64
 75225        74
 75226       166
 75227       193
 75228       102
 75229       118
 75230       112
 75231        45
 75232       109
 75233        55
 75234         4
 75235       163
 75236       193
 75237       100
 75238        17
 75240        16
 75241       338
 75243       100
 75244         9
 75246        57
 75247        68
 75248        39
 75249        22
 75251         

In [81]:
#31 How many private owners with five or more vacant properties in the City of Dallas are in each council district?

private_5_district=owner_vacant_property_counts.groupby('Council District').size()

In [82]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"] = {
    'question_number': 31,
    'value': private_5_district.to_frame(name = 'count')
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"]

{'question_number': 31,
 'value':                   count
 Council District       
 District 1          421
 District 10         131
 District 11         139
 District 12          39
 District 13         194
 District 14         254
 District 2          635
 District 3          589
 District 4         1046
 District 5          426
 District 6         1463
 District 7         1207
 District 8          695
 District 9          105}

In [83]:
#32 How many churches own vacant properties in the City of Dallas?

church_owners = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes']

num_church_properties = church_owners.shape[0]



In [84]:

results["How many churches own vacant properties in the City of Dallas?"] = {
    'question_number': 32,
    'value': pd.Series(num_church_properties).to_frame(name='count')
}

results["How many churches own vacant properties in the City of Dallas?"] 

{'question_number': 32,
 'value':    count
 0     35}

In [85]:
#33 How many churches owned vacant properties in the City of Dallas are in each zip code?

church_prop_zip = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
church_prop_zip.index = church_prop_zip.index.astype(int)


In [86]:

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 33,
    'value': church_prop_zip.to_frame(name = 'count')
}

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] 

{'question_number': 33,
 'value':           count
 Zip Code       
 75203         2
 75211         2
 75212         4
 75215         2
 75216         6
 75217         3
 75223         1
 75224         2
 75226         2
 75227         3
 75228         1
 75238         1
 75241         6}

In [87]:
#34 How many are in each council district

church_prop_district = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Council District').size()


In [88]:

results["How many churches owned vacant properties in each council district"] = {
    'question_number': 34,
    'value': church_prop_district.to_frame(name = 'count')
}

results["How many churches owned vacant properties in each council district"]

{'question_number': 34,
 'value':                   count
 Council District       
 District 1            1
 District 2            1
 District 3            1
 District 4            8
 District 5            2
 District 6            4
 District 7           10
 District 8            7
 District 9            1}

In [89]:
type(results)

dict

In [90]:
{key: type(value['value'])  for key, value in results.items()}

{'How many vacant lots are in each council district?': pandas.core.frame.DataFrame,
 'How long have the vacant lots in each council district been vacant?': pandas.core.frame.DataFrame,
 'How many vacant lots in each council district are zoned single family residential, commercial and multifamily?': pandas.core.frame.DataFrame,
 'How many vacant structures are in each council district?': pandas.core.frame.DataFrame,
 'How long have the vacant structures in each council district been vacant?': pandas.core.frame.DataFrame,
 'How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)': pandas.core.frame.DataFrame,
 'How many vacant lots are in each zip code?': pandas.core.frame.DataFrame,
 'How long have the vacant lots in each zip code been vacant?': pandas.core.frame.DataFrame,
 'How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?': pandas.core.frame.DataFrame,
 '

# Save result to disk as Pickle object

In [93]:
pkl_filename = "vacant_lots_analysis_results.pkl"
pickle.dump( results, open( pkl_filename, "wb" ) )

In [94]:

# Sample code for loading it back into memory
# results = pickle.load( open( pkl_filename, "rb" ) )

# Export to Excel

In [115]:
excel_output_file = "vacant_lots_analysis_export.xlsx"

with pd.ExcelWriter(excel_output_file) as writer:

    for key, value in results.items():
        sheet_name = f"Q-{value['question_number']}"
        print(f"Exporting results for {sheet_name}")  
        question_label = pd.Series(key).to_frame()
        question_label.to_excel(writer, sheet_name=sheet_name, startcol=0, startrow=0, index = False, header = False) 
        value['value'].to_excel(writer, sheet_name=sheet_name, startcol=3, startrow=2) 
        
        #workbook  = writer.book
        #worksheet = writer.sheets[sheet_name]
        #wrap_format = workbook.add_format({'text_wrap': True})
        #worksheet.set_column(excel_header, None, wrap_format)
    

Exporting results for Q-1
Exporting results for Q-2
Exporting results for Q-3
Exporting results for Q-4
Exporting results for Q-5
Exporting results for Q-6
Exporting results for Q-7
Exporting results for Q-8
Exporting results for Q-9
Exporting results for Q-10
Exporting results for Q-11
Exporting results for Q-12
Exporting results for Q-13
Exporting results for Q-14
Exporting results for Q-15
Exporting results for Q-16
Exporting results for Q-17
Exporting results for Q-18
Exporting results for Q-19
Exporting results for Q-20
Exporting results for Q-21
Exporting results for Q-22
Exporting results for Q-23
Exporting results for Q-24
Exporting results for Q-25
Exporting results for Q-26
Exporting results for Q-27
Exporting results for Q-28
Exporting results for Q-29
Exporting results for Q-30
Exporting results for Q-31
Exporting results for Q-32
Exporting results for Q-33
Exporting results for Q-34


In [111]:
help(writer.book.add_named_style)

Help on method add_named_style in module openpyxl.workbook.workbook:

add_named_style(style) method of openpyxl.workbook.workbook.Workbook instance
    Add a named style



In [114]:
workbook.

['Normal']